In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Dataset stores the samples and their corresponding labels.

# Download training and test data from open datasets.
training_data = datasets.FashionMNIST(
    root="data", train=True, download=True, transform=ToTensor())
test_data = datasets.FashionMNIST(
    root="data", train=False, download=True, transform=ToTensor())

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
batch_size = 64

# DataLoader wraps an iterable around the Dataset.
# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size)
test_dataloader = DataLoader(test_data, batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")  # (batch_size, channel, height, width)
    print(f"Shape of y: {y.shape} {y.dtype}")  # (batch_size, )
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Creating Models
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10))  # classes: 10
    
    def forward(self, x):
        x = self.flatten(x)  # input layer
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [4]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()  # set trainable ??
    for batch_i, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)  # len: batch_size: 64

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_i % 100 == 0:
            loss, current = loss.item(), batch_i * len(X)
            print(f"loss: {loss:>7f}, [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    n_batches = len(dataloader)
    model.eval()  # set evaluation ??
    
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= n_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [5]:
# hyper parameters
lr = 1e-3  # learning rate, 0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
------
loss: 2.306894, [    0/60000]
loss: 2.298295, [ 6400/60000]
loss: 2.276587, [12800/60000]
loss: 2.269587, [19200/60000]
loss: 2.250655, [25600/60000]
loss: 2.219145, [32000/60000]
loss: 2.229522, [38400/60000]
loss: 2.189734, [44800/60000]
loss: 2.189353, [51200/60000]
loss: 2.146918, [57600/60000]
Test Error: 
 Accuracy: 39.2%, Avg loss: 2.151695 

Epoch 2
------
loss: 2.164797, [    0/60000]
loss: 2.163285, [ 6400/60000]
loss: 2.100847, [12800/60000]
loss: 2.115141, [19200/60000]
loss: 2.060230, [25600/60000]
loss: 1.994923, [32000/60000]
loss: 2.034085, [38400/60000]
loss: 1.949426, [44800/60000]
loss: 1.954835, [51200/60000]
loss: 1.869219, [57600/60000]
Test Error: 
 Accuracy: 53.0%, Avg loss: 1.880553 

Epoch 3
------
loss: 1.920113, [    0/60000]
loss: 1.899479, [ 6400/60000]
loss: 1.773905, [12800/60000]
loss: 1.809727, [19200/60000]
loss: 1.694933, [25600/60000]
loss: 1.648467, [32000/60000]
loss: 1.678424, [38400/60000]
loss: 1.579825, [44800/60000]
loss: 1.597